### Play with networktables to see how the API is organized  - 20220313 CJH

In [2]:
import time
import networktables
import pandas as pd
from _pyntcore._ntcore import NetworkTableType
import requests

In [3]:
ntinst = networktables.NetworkTablesInstance.getDefault()

#### can give nt multiple servers to try

In [4]:
servers = ["127.0.0.1", "10.24.29.2"] #, "roboRIO-2429-FRC.local"]  # need to add the USB one here
ntinst.startClient(servers=servers)

#### if it connects you can get the connection info - may have to wait a few seconds though

In [5]:
connections = ntinst.getConnections()

In [6]:
print(f'ID:{connections[0].remote_id}\tIP:{connections[0].remote_ip}')

ID:Robot	IP:10.24.29.2


#### looks like all entries are given their own subtable

In [7]:
table = ntinst.getGlobalTable()  # also NetworkTables.getTable('/')
table

In [8]:
test_obj = table
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

PATH_SEPARATOR_CHAR, addEntryListener, addSubTableListener, basenameKey, clearFlags, clearPersistent, containsKey, containsSubTable
delete, getBoolean, getBooleanArray, getEntry, getFlags, getHierarchy, getInstance, getKeys
getNumber, getNumberArray, getPath, getRaw, getString, getStringArray, getSubTable, getSubTables
getValue, isPersistent, loadEntries, normalizeKey, putBoolean, putBooleanArray, putNumber, putNumberArray
putRaw, putString, putStringArray, putValue, removeEntryListener, removeTableListener, saveEntries, setDefaultBoolean
setDefaultBooleanArray, setDefaultNumber, setDefaultNumberArray, setDefaultRaw, setDefaultString, setDefaultStringArray, setDefaultValue, setFlags
setPersistent, 

In [9]:
subtables = table.getSubTables()
type(subtables)

list

In [10]:
len(subtables)

95

In [11]:
set(subtables)

{'CameraPublisher', 'FMSInfo', 'LiveWindow', 'SmartDashboard', 'photonvision'}

In [12]:
s = table.getSubTable('photonvision')

In [13]:
set(s.getSubTables())

{'Microsoft_LifeCam_HD-3000'}

In [34]:
entry = s.getEntry('Microsoft_LifeCam_HD-3000/targetPose')

In [36]:
pose = entry.getDoubleArray([])

In [37]:
pose

[3.0850190290684543,
 0.3003502548711148,
 -0.06576292445761922,
 -0.10251162487105833,
 0.28107288153338994,
 -0.16338905977669826,
 0.9401028758482228]

In [41]:
entry = s.getEntry('Microsoft_LifeCam_HD-3000/rawBytes')

In [42]:
raw = entry.getRaw([])

In [43]:
raw

b'@j\x08 \xc4\x9b\xa5\xe3\x03\xc0\x17\xf0\x06\xd9F\x11c?\xde\xd7zH\xabWk?\xd1\xea\xaa\xaa\xaa\xaa\xab\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x08\x98\xd1\x99\xc6rD?\xd3+\xc9\xb9\xfeG\xde\xbf\xb0\xf0C\x1d\x19\x80\x80\xbf\xb9\xd3\xbdx\xed\xad\xa0?\xd1\xf8\xcc\x00\xd8\xad\x0c\xbf\xc4\xdc\x0c[\x97x\x88?\xee\x18\x01\xd0\x90 \xb2@\x08\x9a \xf3\x9c\xbb\x87?\xd37n~\x88ph\xbf\xb1Uw\xce\xeah\xa0?\x8bt\x96\xd6\x8e\xb90\xbf\xce\xf5e\xd8\xd4Q)\xbf\xc7\xf7-<oH\xa2?\xeev\x9f\xb0[\xeb\xa8?\xadw|\x86\xd7\xf6%@]\xbb\r]K\x06\xb5@^\x0c\xdd\xb8\x19\xfbY@^\xfb\r]\x19\x17i@Z\xcc\xdd\xba\xe1\x9d\x13@a`\x00\x11Z|\xa6@\\@\x00\x07\xe6\x04\xa7@`\xc0\x00\x11stL@_\x80\x00\x05\x1eb\xed@0\x9b\xdf\xe66\xd3n@\x15n\xe7\xe1\xde%/?\xd1@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02@\n\x85\x05\xce\x03N_\xbf\xeb\xf3^\x12\x96\x91\xc2?\xc7U\x86V/\xd5\x90\xbf\xb0g\xc8\x9c\x12\xf9\xc4?\x8b\x01\xb7X\x1c\x00`?\xefzM*\x95T\xe2?\xc5t[\xf9\x90\xb9\n@\n\xb1\xe5P=n\xbe\xbf\xec\x1c\x88\x9cz\x06\xbc?\xc7j\

In [48]:
import struct

In [54]:
struct.iter_unpack('s', raw)

In [52]:
len(raw)/4

170.5

In [13]:
unique = list(set(subtables))
frequency = {}
for item in unique:
    frequency[item] = subtables.count(item)
print("Frequency of items: ", frequency)

Frequency of items:  {'FMSInfo': 9, 'photonvision': 17, 'SmartDashboard': 29, 'CameraPublisher': 12, 'LiveWindow': 28}


In [16]:
r = table.getSubTable('Ramsete')

#### get all entries - not sure if this is the fastest way to do it

In [17]:
entries = ntinst.getEntries('/', types=0)

In [18]:
sorted_tree = sorted([e.getName() for e in entries])
sorted_tree[::20]

['/BallCam//blue/distance',
 '/CameraPublisher/Ballcam Processed/description',
 '/CameraPublisher/FosCAM/streams',
 '/CameraPublisher/MS LifeCam/PropertyInfo/brightness/min',
 '/CameraPublisher/MS LifeCam/PropertyInfo/power_line_frequency/default',
 '/CameraPublisher/MS LifeCam/PropertyInfo/zoom_absolute/default',
 '/CameraPublisher/MS LifeCam/RawPropertyInfo/raw_exposure_absolute/step',
 '/CameraPublisher/Shootercam CV Image Source/streams',
 '/LiveWindow/Ungrouped/AnalogInput[0]/Value',
 '/LiveWindow/Ungrouped/Indexer/.name',
 '/LiveWindow/Ungrouped/Pneumatics/.name',
 '/LiveWindow/Ungrouped/Solenoid[0,1]/Value',
 '/LiveWindow/Ungrouped/class frc2::SubsystemBase/.default',
 '/Rev/kFF_vel',
 '/Rev_Climber/kIz_pos',
 '/SmartDashboard//sim/field_rot',
 '/SmartDashboard/AutoRotateSparkmax/.name',
 '/SmartDashboard/CompressorToggle/.name',
 '/SmartDashboard/Reset Encoders/.name',
 '/SmartDashboard/autonomous routines/.controllable',
 '/SmartDashboard/drive_pose',
 '/SmartDashboard/path ve

#### why do i have to take what was a table and get a flat list?

In [19]:
nt_dict = {}
for subtable in sorted(set(subtables)):
    child = [entry[len(subtable)+2:] for entry in sorted_tree if subtable in entry]
    nt_dict.update({subtable:child})
#nt_dict

#### data dump from networktables

In [20]:
d = {}
for ix, item in enumerate(sorted_tree):
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    # value = val.getDouble() if val.isDouble() else val.getBoolean() if val.isBoolean else val.getString if val.isString else ''
    value = entry_val.value()
   # print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  RPC:{entry_val.isRaw()} Levels:{name[1:].split("/")}')
    d.update({entry.getName(): {'value':str(value), 'age':int(time.time() - entry_val.time()/1E6),  'type': entry.getType(),  'RPC':entry_val.isRaw(), 'levels':name[1:].split("/")}})
df = pd.DataFrame(d).transpose()
pd.set_option('display.max_rows', df.shape[0]+1)
display(df)

,value,age,type,RPC,levels
/BallCam//blue/distance,0.0,2,NetworkTableType.kDouble,False,"[BallCam, , blue, distance]"
/BallCam//blue/rotation,0.0,2,NetworkTableType.kDouble,False,"[BallCam, , blue, rotation]"
/BallCam//blue/strafe,0.0,2,NetworkTableType.kDouble,False,"[BallCam, , blue, strafe]"
/BallCam//blue/targets,0.0,2,NetworkTableType.kDouble,False,"[BallCam, , blue, targets]"
/BallCam//green/distance,0.2695029,0,NetworkTableType.kDouble,False,"[BallCam, , green, distance]"
/BallCam//green/rotation,15.812499999999998,0,NetworkTableType.kDouble,False,"[BallCam, , green, rotation]"
/BallCam//green/strafe,0.0,2,NetworkTableType.kDouble,False,"[BallCam, , green, strafe]"
/BallCam//green/targets,1.0,0,NetworkTableType.kDouble,False,"[BallCam, , green, targets]"
/BallCam//red/distance,1.872051196361601,2,NetworkTableType.kDouble,False,"[BallCam, , red, distance]"
/BallCam//red/rotation,-6.637500000000003,2,NetworkTableType.kDouble,False,"[BallCam, , red, rotation]"


In [21]:
ntdict = {}
for item in sorted_tree:
    entry = ntinst.getEntry(item)
    entry_val = entry.getValue()
    name = entry.getName()
    value = entry_val.value()
    ntdict.update({name:{'nt_path': entry.getName(), 'value':value, 'entry':entry, 'type':entry.getType()}})
    #print(f'Name: {entry.getName():60s} Value:{str(value):20s} \t Age: {int(time.time() - entry_val.time()/1E6):6d}  Type: {entry.getType()}  Levels:{name[1:].split("/")}')
df = pd.DataFrame(ntdict).transpose()
display(df)

,nt_path,value,entry,type
/BallCam//blue/distance,/BallCam//blue/distance,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//blue/rotation,/BallCam//blue/rotation,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//blue/strafe,/BallCam//blue/strafe,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//blue/targets,/BallCam//blue/targets,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//green/distance,/BallCam//green/distance,0.269503,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//green/rotation,/BallCam//green/rotation,15.984375,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//green/strafe,/BallCam//green/strafe,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//green/targets,/BallCam//green/targets,0.0,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//red/distance,/BallCam//red/distance,1.872051,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble
/BallCam//red/rotation,/BallCam//red/rotation,-6.6375,<_pyntcore._ntcore.NetworkTableEntry object at...,NetworkTableType.kDouble


#### investigate an entry

In [23]:
e = s.getEntry('hasTarget')

NameError: name 'e' is not defined

In [14]:
t = ntinst.getEntry('/photonvision/Microsoft_LifeCam_HD-3000/')

In [15]:
t

In [16]:
test_obj = t
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

Flags, addListener, clearFlags, clearPersistent, delete, exists, forceSetBoolean, forceSetBooleanArray
forceSetDouble, forceSetDoubleArray, forceSetRaw, forceSetString, forceSetStringArray, forceSetValue, getBoolean, getBooleanArray
getDouble, getDoubleArray, getFlags, getHandle, getInfo, getInstance, getLastChange, getName
getRaw, getString, getStringArray, getType, getValue, isPersistent, removeListener, setBoolean
setBooleanArray, setDefaultBoolean, setDefaultBooleanArray, setDefaultDouble, setDefaultDoubleArray, setDefaultRaw, setDefaultString, setDefaultStringArray
setDefaultValue, setDouble, setDoubleArray, setFlags, setPersistent, setRaw, setString, setStringArray
setValue, value, 

In [17]:
t.getName()

'/photonvision/ledMode'

In [18]:
t.getType()

<NetworkTableType.kDouble: 2>

In [27]:
type(t.getType())

_pyntcore._ntcore.NetworkTableType

In [28]:
from _pyntcore._ntcore import NetworkTableType

In [29]:
NetworkTableType.kBoolean

<NetworkTableType.kBoolean: 1>

In [30]:
t.getDoubleArray([1,1,1,])

[7.647, 1.935, -180.0]

In [31]:
info = t.getInfo()

In [32]:
[item for item in dir(info) if '_' not in item ]

['entry', 'flags', 'name', 'type']

In [33]:
t.getType()

<NetworkTableType.kDoubleArray: 32>

In [34]:
t.getLastChange()

1648662074472729

In [35]:
val = t.getValue()

In [36]:
test_obj = val
attribs = [attrib for attrib in dir(test_obj) if '__' not in attrib]
_ = [print(attrib, end=', ') if (ix+1)%8!=0 else print(attrib, end='\n') for ix, attrib in enumerate(attribs) ]

getBoolean, getBooleanArray, getDouble, getDoubleArray, getFactoryByType, getRaw, getRpc, getString
getStringArray, isBoolean, isBooleanArray, isDouble, isDoubleArray, isRaw, isRpc, isString
isStringArray, isValid, last_change, makeBoolean, makeBooleanArray, makeDouble, makeDoubleArray, makeRaw
makeRpc, makeString, makeStringArray, makeValue, time, type, value, 

In [37]:
val.getDouble()

8.91731538824e-312

In [38]:
val.last_change()

1648662074472729

In [39]:
val.time()

1648662074472729

In [40]:
val.time??

Docstring:
time(self: _pyntcore._ntcore.Value) -> int

Get the creation time of the value.

:returns: The time, in the units returned by nt::Now().
Type:      method


In [41]:
val.value()

[7.647, 1.935, -180.0]

In [42]:
t2 = ntinst.getEntry('/SmartDashboard/autonomous routines/options')

In [43]:
val = t2.getValue()
val.getStringArray()

['2 ball only', '3 ball terminal', '4 ball lower', 'Ramsete Test']

In [44]:
t2.getStringArray([])

['2 ball only', '3 ball terminal', '4 ball lower', 'Ramsete Test']

In [45]:
type(val.getStringArray())

list

In [46]:
# generate the dictionary
d={}
levels = [s[1:].split('/') for s in sorted_tree]
for path in levels:
    current_level = d
    for part in path:
        if part not in current_level:
            current_level[part] = {}
        current_level = current_level[part]
#d

In [47]:
levels[100]

['CameraPublisher', 'MS LifeCam', 'PropertyInfo', 'zoom_absolute', 'default']

In [48]:
d['LiveWindow']['Ungrouped']['navX-Sensor[4]']['Value']

{}

In [49]:
for item in sorted_tree:
    value = ntinst.getEntry(item).getValue().value()
    levels = item[1:].split('/')
    if len(levels) == 2:
        d[levels[0]][levels[1]] = value
    elif len(levels) == 3:
        d[levels[0]][levels[1]][levels[2]] = value
    elif len(levels) == 4:
        d[levels[0]][levels[1]][levels[2]][levels[3]] = value

In [50]:
d

{'BallCam': {'': {'blue': {'distance': 0.0,
    'rotation': 0.0,
    'strafe': 0.0,
    'targets': 0.0},
   'green': {'distance': 0.2770836,
    'rotation': 15.812499999999998,
    'strafe': 0.0,
    'targets': 1.0},
   'red': {'distance': 1.872051196361601,
    'rotation': -6.637500000000003,
    'strafe': -0.2163853185071734,
    'targets': 0.0}},
  'frames': 30700.0},
 'CameraPublisher': {'Ballcam CV Image Source': {'connected': True,
   'description': '',
   'mode': '320x240 MJPEG 20 fps',
   'modes': ['320x240 MJPEG 20 fps'],
   'source': 'cv:',
   'streams': []},
  'Ballcam Processed': {'connected': False,
   'description': '',
   'mode': '0x0 Unknown 0 fps',
   'modes': ['640x480 MJPEG 30 fps',
    '320x240 MJPEG 30 fps',
    '160x120 MJPEG 30 fps'],
   'source': 'ip:http://127.0.0.1:1186/?action=stream',
   'streams': ['mjpg:http://127.0.0.1:1186/?action=stream']},
  'FosCAM': {'Property': {'brightness': 50.0},
   'PropertyInfo': {'brightness': {'default': {},
     'max': {},
 

In [51]:
def depth(d):
    if isinstance(d, dict):
        return 1 + (max(map(depth, d.values())) if d else 0)
    return 0

In [52]:
depth(d)

6

In [53]:
widget_dict = {'qlabel_camera_view': {'widget':'self.qlabel_camera_view', 'table':''},
        'qlabel_climber_indicator':{'widget':'self.qlabel_climber_indicator', 'table':''},
        }

In [54]:
for key, value in widget_dict.items():
    print(key, value)

qlabel_camera_view {'widget': 'self.qlabel_camera_view', 'table': ''}
qlabel_climber_indicator {'widget': 'self.qlabel_climber_indicator', 'table': ''}


In [1]:
import requests

In [28]:
def url_checker(url):
    try:
        #Get Url
        get = requests.get(url, timeout=0.25)
        # if the request succeeds 
        if get.status_code == 200:
            return(f"{url}: is reachable")
        else:
            return(f"{url}: is Not reachable, status_code: {get.status_code}")

    #Exception
    except requests.exceptions.RequestException as e:
        # print URL with Errs
        print(f"{url}: is Not reachable \nErr: {e}")
        # raise SystemExit(f"{url}: is Not reachable \nErr: {e}")

In [13]:
url = 'http://10.24.29.12:1187/stream.mjpg'

In [17]:
import urllib.request
def check_url(url):
    try:
        code = urllib.request.urlopen(url, timeout=0.1).getcode()
        print(f'return code is {code}')
        if code == 200:
            return True
    except Exception as e:
        print(f'Failed: {e}')
    return False

In [18]:
check_url(url)

return code is 200


True